In [55]:
import pandas as pd
import json
import plotly.express as px

# fix plotly express et Visual Studio Code
import plotly.io as pio
pio.renderers.default = "notebook_connected"




# Import data

In [56]:
f = open('Tømløra-158491.json', encoding="utf8")

In [57]:
data = json.load(f)

# Dataframe
## Exploration

In [58]:
df = pd.DataFrame.from_dict(data, orient='index')

In [59]:
df

,0
command,HubUserLogin
ret_code,0
wizard_info,"{'wizard_id': 158491, 'wizard_name': 'Tømløra'..."
scenario_list,"[{'region_id': 1, 'difficulty': 1, 'cleared': ..."
defense_unit_list,"[{'pos_id': 1, 'unit_id': 11259820848}, {'pos_..."
...,...
tvaluelocal,1649482037
tzone,Europe/Berlin
server_id,6
server_endpoint,summonerswar-eu-lb


In [60]:
df = df.transpose()

In [61]:
df.shape
print(f'{df.shape[0]} lignes et {df.shape[1]} colonnes')

1 lignes et 139 colonnes


In [62]:
df

,command,ret_code,wizard_info,scenario_list,defense_unit_list,server_arena_defense_unit_list,quest_active,quest_rewarded,event_id_list,building_list,...,lobby_map_disable_list,tvaluelocal_next_monday,wizard_id,ts_val,tvalue,tvaluelocal,tzone,server_id,server_endpoint,unit_storage_list
0,HubUserLogin,0,"{'wizard_id': 158491, 'wizard_name': 'Tømløra'...","[{'region_id': 1, 'difficulty': 1, 'cleared': ...","[{'pos_id': 1, 'unit_id': 11259820848}, {'pos_...","[{'unit_id': 11259820848, 'pos_id': 1}, {'unit...","[{'quest_id': 45, 'is_completed': 0, 'conditio...","[2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 15, 16,...","[2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 15, 17, ...","[{'building_id': 2309392, 'wizard_id': 158491,...",...,[],1649628000,158491,2122218797,1649507237,1649482037,Europe/Berlin,6,summonerswar-eu-lb,"[{'unit_master_id': 10101, 'class': 3, 'quanti..."


# Exploration colonnes restantes

{'craft_item_id': 144593998,
  'wizard_id': 158491,
  'craft_type': 2,
  'craft_type_id': 150404,
  'sell_value': 16000,
  'amount': 37},
  
  Will heroique atk%
  
  15 = will
  04 = hero
  04 = atk%
  
   {'craft_item_id': 274591133,
  'wizard_id': 158491,
  'craft_type': 2,
  'craft_type_id': 150204,
  'sell_value': 16000,
  'amount': 19},
  
  15 = will
  02 = % hp
  04 = heroique
  
   {'craft_item_id': 275119059,
  'wizard_id': 158491,
  'craft_type': 2,
  'craft_type_id': 150604,
  'sell_value': 16000,
  'amount': 24},
  
  15 = will
  06 = def %
  04 = hero

In [63]:
df_final = df['rune_craft_item_list'] #après gros nettoyage (cleané ici), la variable qui nous intéresse

In [64]:
df_final[0][1]

{'craft_item_id': 4112939,
 'wizard_id': 158491,
 'craft_type': 1,
 'craft_type_id': 31105,
 'sell_value': 37000,
 'amount': 2}

# craft_type

In [65]:
CRAFT_TYPE_MAP = {
    1:'Enchant_gem',
    2:'Grindstone',
    3:'Gemme_immemoriale',
    4:'Grindstone_immemoriale',
    5:'Ancienne_gemme',
    6:'Ancienne_grindstone',
}


# Craft_type_id
## Rune

In [66]:
    COM2US_TYPE_MAP = {
        1: 'TYPE_ENERGY',
        2: 'TYPE_GUARD',
        3: 'TYPE_SWIFT',
        4: 'TYPE_BLADE',
        5: 'TYPE_RAGE',
        6: 'TYPE_FOCUS',
        7: 'TYPE_ENDURE',
        8: 'TYPE_FATAL',
        10: 'TYPE_DESPAIR',
        11: 'TYPE_VAMPIRE',
        13: 'TYPE_VIOLENT',
        14: 'TYPE_NEMESIS',
        15: 'TYPE_WILL',
        16: 'TYPE_SHIELD',
        17: 'TYPE_REVENGE',
        18: 'TYPE_DESTROY',
        19: 'TYPE_FIGHT',
        20: 'TYPE_DETERMINATION',
        21: 'TYPE_ENHANCE',
        22: 'TYPE_ACCURACY',
        23: 'TYPE_TOLERANCE',
        99: 'IMMEMORIALE'
    }

## stat

In [67]:
    COM2US_STAT_MAP = {
        1: 'STAT_HP',
        2: 'STAT_HP_PCT',
        3: 'STAT_ATK',
        4: 'STAT_ATK_PCT',
        5: 'STAT_DEF',
        6: 'STAT_DEF_PCT',
        8: 'STAT_SPD',
        9: 'STAT_CRIT_RATE_PCT',
        10: 'STAT_CRIT_DMG_PCT',
        11: 'STAT_RESIST_PCT',
        12: 'STAT_ACCURACY_PCT',
    }

## quality

In [68]:
    COM2US_QUALITY_MAP = {
        1: 'QUALITY_NORMAL',
        2: 'QUALITY_MAGIC',
        3: 'QUALITY_RARE',
        4: 'QUALITY_HERO',
        5: 'QUALITY_LEGEND',
        # Original quality values
        11: 'QUALITY_NORMAL',
        12: 'QUALITY_MAGIC',
        13: 'QUALITY_RARE',
        14: 'QUALITY_HERO',
        15: 'QUALITY_LEGEND',
    }

In [69]:
nb_boucle = len(df_final[0])
nb_boucle

503

In [70]:
# on va map avec une boucle
for i in range(0, nb_boucle):
    objet = str(df_final[0][i]['craft_type_id'])
    nb_chiffre = len(objet)
        
    type = int(df_final[0][i]['craft_type'])
    rune = int(objet[:nb_chiffre-4])
    stat = int(objet[nb_chiffre-4:nb_chiffre-2])
    quality = int(objet[nb_chiffre-2:nb_chiffre])
    
    df_final[0][i]['type'] = CRAFT_TYPE_MAP[type]
    df_final[0][i]['rune'] = COM2US_TYPE_MAP[rune]
    df_final[0][i]['stat'] = COM2US_STAT_MAP[stat]
    df_final[0][i]['quality'] = COM2US_QUALITY_MAP[quality]
    
    
print("Exemple découpage : item : " + str(objet) + "\n rune : " + str(rune) + "\n stat : " + str(stat) + "\n quality : " + str(quality))
    

Exemple découpage : item : 150814
 rune : 15
 stat : 8
 quality : 14


In [71]:
df_final[0][1]

{'craft_item_id': 4112939,
 'wizard_id': 158491,
 'craft_type': 1,
 'craft_type_id': 31105,
 'sell_value': 37000,
 'amount': 2,
 'type': 'Enchant_gem',
 'rune': 'TYPE_SWIFT',
 'stat': 'STAT_RESIST_PCT',
 'quality': 'QUALITY_LEGEND'}

In [72]:
df = pd.DataFrame(df_final)
df

,rune_craft_item_list
0,"[{'craft_item_id': 4112928, 'wizard_id': 15849..."


In [73]:
import ast
def extraire_variables_imbriquees(df, colonne):
    df[colonne] = [ast.literal_eval(str(item)) for index, item in df[colonne].iteritems()]

    df = pd.concat([df.drop([colonne], axis=1), df[colonne].apply(pd.Series)], axis=1)
    return df

In [74]:
df = extraire_variables_imbriquees(df,'rune_craft_item_list') #découpe le dictionnaire imbriqué en un dict = une variable

In [75]:
# on refait pour sortir toutes les variables de chaque dict.... et on concatène pour n'avoir qu'un seul dataframe
df_combine = extraire_variables_imbriquees(df,0)
df_combine = df_combine[['craft_item_id', 'wizard_id', 'craft_type', 'craft_type_id', 'sell_value', 'amount' ,'type', 'rune', 'stat', 'quality']]
for i in range(1, len(df.columns)):
    df_combine2 = extraire_variables_imbriquees(df, i)
    df_combine2 = df_combine2[['craft_item_id', 'wizard_id', 'craft_type', 'craft_type_id', 'sell_value', 'amount' ,'type', 'rune', 'stat', 'quality']]
    df_combine = pd.concat([df_combine,df_combine2])


In [76]:
df_combine

,craft_item_id,wizard_id,craft_type,craft_type_id,sell_value,amount,type,rune,stat,quality
0,4112928,158491,1,11204,30000,3,Enchant_gem,TYPE_ENERGY,STAT_ACCURACY_PCT,QUALITY_HERO
0,4112939,158491,1,31105,37000,2,Enchant_gem,TYPE_SWIFT,STAT_RESIST_PCT,QUALITY_LEGEND
0,4112942,158491,1,40604,30000,6,Enchant_gem,TYPE_BLADE,STAT_DEF_PCT,QUALITY_HERO
0,4112955,158491,1,70804,30000,5,Enchant_gem,TYPE_ENDURE,STAT_SPD,QUALITY_HERO
0,4112969,158491,1,101204,30000,6,Enchant_gem,TYPE_DESPAIR,STAT_ACCURACY_PCT,QUALITY_HERO
...,...,...,...,...,...,...,...,...,...,...
0,325714194,158491,6,180814,19200,4,Ancienne_grindstone,TYPE_DESTROY,STAT_SPD,QUALITY_HERO
0,334292589,158491,6,20115,27600,1,Ancienne_grindstone,TYPE_GUARD,STAT_HP,QUALITY_LEGEND
0,343007741,158491,6,150614,19200,2,Ancienne_grindstone,TYPE_WILL,STAT_DEF_PCT,QUALITY_HERO
0,350683564,158491,6,110414,19200,1,Ancienne_grindstone,TYPE_VAMPIRE,STAT_ATK_PCT,QUALITY_HERO


In [77]:
fig = px.histogram(df_combine, x="rune", y="amount", histfunc='sum', text_auto=True, title="Par type de rune").update_xaxes(categoryorder="total descending")
fig.show()

# Stats

In [78]:
fig = px.pie(df_combine, values="amount", names="type", title="Répartition inventaire")
fig.show()